<a href="https://colab.research.google.com/github/fmarcelneves/Rental-price-analysis-and-forecast-Curitiba-PR/blob/main/Data_exploration_Curitiba_rent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages 

In [1]:
%%capture
!pip install geopandas
!pip install outlier-removal-101703289
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from outlier_removal.outlier import remove_outlier
!pip install unidecode
import unidecode

In [2]:
%%capture
!pip install plotly --upgrade
from plotly import __version__
print (__version__)
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly as py
#import plotly.figure_factory as ff
from plotly import tools
#import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell

# Import data 

Import extracted data 

In [3]:
url = 'https://raw.githubusercontent.com/fmarcelneves/Rental-price-analysis-and-forecast-Curitiba-PR/main/data_mercado_livre_ap_curitiba_rent.csv'
da_ml = pd.read_csv(url)
#da_ml = pd.read_csv('dados_mercado_livre_curitiba.csv')

Import geodata

In [4]:
!wget 'http://ippuc.org.br/geodownloads/SHAPES/DIVISA_DE_BAIRROS.zip'
!unzip '/content/DIVISA_DE_BAIRROS.zip'

--2021-02-24 18:46:07--  http://ippuc.org.br/geodownloads/SHAPES/DIVISA_DE_BAIRROS.zip
Resolving ippuc.org.br (ippuc.org.br)... 201.11.35.17
Connecting to ippuc.org.br (ippuc.org.br)|201.11.35.17|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337191 (329K) [application/x-zip-compressed]
Saving to: ‘DIVISA_DE_BAIRROS.zip’

DIVISA_DE_BAIRROS.z 100%[===================>] 329.29K   312KB/s    in 1.1s    

2021-02-24 18:46:09 (312 KB/s) - ‘DIVISA_DE_BAIRROS.zip’ saved [337191/337191]

Archive:  /content/DIVISA_DE_BAIRROS.zip
  inflating: DIVISA_DE_BAIRROS.shx   
  inflating: DIVISA_DE_BAIRROS.dbf   
  inflating: DIVISA_DE_BAIRROS.prj   
  inflating: DIVISA_DE_BAIRROS.sbn   
  inflating: DIVISA_DE_BAIRROS.sbx   
  inflating: DIVISA_DE_BAIRROS.shp   


Import IDHM data 

In [13]:
url = 'https://raw.githubusercontent.com/fmarcelneves/Rental-price-analysis-and-forecast-Curitiba-PR/main/IDMH_Bairros_Curitiba.xlsx'
idhm = pd.read_excel(url)
idhm.head()

,NOME_UDH,NOME_MUN,ANO,IDHM
0,Abranches,Curitiba (PR),2010,0.800
1,Água Verde,Curitiba (PR),2010,0.946
2,Ahú,Curitiba (PR),2010,0.929
3,Alto Boqueirão,Curitiba (PR),2010,0.750
4,Alto da Glória,Curitiba (PR),2010,0.949


# Filtering data 

In [ ]:
da_ml = remove_outlier(da_ml)

/usr/local/lib/python3.6/dist-packages/outlier_removal/outlier.py:24: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.6/dist-packages/outlier_removal/outlier.py:24: RuntimeWarning:

invalid value encountered in less



In [ ]:
pd.options.display.float_format = "{:.2f}".format

In [ ]:
da_ml.replace(',','.', regex=True,inplace=True)
da_ml['areas'] = da_ml['areas'].astype(float)

# Data exploration

In [ ]:
da_ml.describe().T

,count,mean,std,min,25%,50%,75%,max
precos,811.00,1303.43,615.21,2.00,850.00,1170.00,1600.00,3250.00
quartos,789.00,1.89,0.84,1.00,1.00,2.00,3.00,5.00
areas,802.00,64.75,49.11,17.00,37.00,53.00,74.00,801.00


In [ ]:
indexNames = da_ml[da_ml['areas'] >= 350].index

In [ ]:
da_ml.drop(indexNames , inplace=True)

In [ ]:
da_ml.describe().T

,count,mean,std,min,25%,50%,75%,max
precos,808.00,1298.97,611.47,2.00,850.00,1160.00,1600.00,3250.00
quartos,786.00,1.88,0.84,1.00,1.00,2.00,3.00,5.00
areas,799.00,62.99,38.08,17.00,37.00,53.00,73.50,300.00


In [ ]:
da_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808 entries, 0 to 915
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   bairro   799 non-null    object 
 1   precos   808 non-null    float64
 2   quartos  786 non-null    float64
 3   areas    799 non-null    float64
dtypes: float64(3), object(1)
memory usage: 31.6+ KB


In [ ]:
da_ml.isnull().sum()

bairro      9
precos      0
quartos    22
areas       9
dtype: int64

In [ ]:
da_ml['bairro'].value_counts()

Reboucas             46
Centro               44
Cidade-Industrial    41
Bigorrilho           39
Cristo-Rei           39
                     ..
Campo-de-Santana      1
Parolin               1
Cachoeira             1
Champagnat            1
Bom-Retiro            1
Name: bairro, Length: 62, dtype: int64

In [ ]:
da_group = da_ml.groupby(['bairro']).median().sort_values('precos', ascending=False)
da_group

,precos,quartos,areas
bairro,,,
Champagnat,2800.00,3.00,193.00
Bigorrilho,2300.00,2.00,67.00
Capao-da-Imbuia,2137.50,3.50,157.00
Jardim-das-Americas,1950.00,1.50,86.00
Batel,1900.00,2.00,88.00
...,...,...,...
Cachoeira,750.00,2.00,45.00
Sao-Lourenco,725.00,1.50,50.00
Tatuquara,675.00,2.00,47.00


In [ ]:
fig = make_subplots(rows=3, cols=1)
preços = go.Histogram(x=da_ml['precos'], name="preços")
quartos = go.Histogram(x=da_ml['quartos'], name="quartos")
áreas = go.Histogram(x=da_ml['areas'], name="áreas")

fig.append_trace(preços, 1, 1)
fig.append_trace(quartos, 2, 1)
fig.append_trace(áreas, 3, 1)
fig.show()

In [ ]:
px.scatter(data_frame=da_ml, x="precos", y="areas")

In [ ]:
px.box(data_frame=da_ml, x="quartos", y="areas")

In [ ]:
px.box(data_frame=da_ml, x="quartos", y="precos")

In [ ]:
da_price = da_group.drop('quartos', 1).drop('areas', 1).reset_index()
da_price['bairro'] = da_price['bairro'].str.capitalize().apply(unidecode.unidecode).replace('-', ' ', regex=True)

In [ ]:
da_areas_precos = da_group.drop('quartos', 1).reset_index()
da_areas_precos['bairro'] = da_areas_precos['bairro'].str.capitalize().apply(unidecode.unidecode).replace('-', ' ', regex=True)

In [ ]:
px.scatter(data_frame=da_areas_precos, x="precos", y="bairro", color='areas')

In [ ]:
da_price

,bairro,precos
0,Champagnat,2800.00
1,Bigorrilho,2300.00
2,Capao da imbuia,2137.50
3,Jardim das americas,1950.00
4,Batel,1900.00
...,...,...
57,Cachoeira,750.00
58,Sao lourenco,725.00
59,Tatuquara,675.00
60,Sitio cercado,616.00


In [ ]:
da_group

,precos,quartos,areas
bairro,,,
Champagnat,2800.00,3.00,193.00
Bigorrilho,2300.00,2.00,67.00
Capao-da-Imbuia,2137.50,3.50,157.00
Jardim-das-Americas,1950.00,1.50,86.00
Batel,1900.00,2.00,88.00
...,...,...,...
Cachoeira,750.00,2.00,45.00
Sao-Lourenco,725.00,1.50,50.00
Tatuquara,675.00,2.00,47.00


In [ ]:
correlations = da_ml.corr()
fig = px.imshow(correlations, labels=dict(color="Correlação"), title="Correlation between numerical variables")
#fig.update_layout(width=800, height=800)
fig.show()

In [ ]:
counts = da_ml['bairro'].value_counts()

In [ ]:
counts.values

array([46, 44, 41, 39, 39, 38, 33, 29, 26, 23, 21, 19, 18, 18, 17, 16, 16,
       16, 15, 15, 14, 14, 14, 13, 12, 12, 12, 12, 11, 11, 11, 10,  9,  9,
        9,  8,  8,  8,  6,  6,  5,  5,  5,  4,  4,  4,  4,  3,  3,  3,  3,
        3,  2,  2,  2,  2,  2,  1,  1,  1,  1,  1])

In [ ]:
#da_ml = da_ml[da_ml['bairro'].isin(counts[counts > 3].index)]

In [ ]:
da_ml['bairro'].value_counts().values

array([46, 44, 41, 39, 39, 38, 33, 29, 26, 23, 21, 19, 18, 18, 17, 16, 16,
       16, 15, 15, 14, 14, 14, 13, 12, 12, 12, 12, 11, 11, 11, 10,  9,  9,
        9,  8,  8,  8,  6,  6,  5,  5,  5,  4,  4,  4,  4,  3,  3,  3,  3,
        3,  2,  2,  2,  2,  2,  1,  1,  1,  1,  1])

In [ ]:
da_ml.groupby('bairro').count()

,precos,quartos,areas
bairro,,,
Abranches,8,8,8
Agua-Verde,38,37,38
Ahu,10,10,10
Alto-Boqueirao,13,13,13
Alto-da-Gloria,12,12,12
...,...,...,...
Tingui,14,14,14
Uberaba,15,15,15
Vila-Izabel,18,17,18


In [ ]:
#da_ml.to_csv('data_rent_ap_curitiba_mercado_livre.csv')

In [ ]:
da_group.head()

,precos,quartos,areas
bairro,,,
Champagnat,2800.00,3.00,193.00
Bigorrilho,2300.00,2.00,67.00
Capao-da-Imbuia,2137.50,3.50,157.00
Jardim-das-Americas,1950.00,1.50,86.00
Batel,1900.00,2.00,88.00


In [ ]:
#da_price = da_group.drop('quartos', 1).drop('areas', 1).reset_index()

In [ ]:
#da_price['bairro'] = da_price['bairro'].str.capitalize().apply(unidecode.unidecode).replace('-', ' ', regex=True)

geo files 

In [ ]:
db = gpd.read_file('DIVISA_DE_BAIRROS.shp')

In [ ]:
db.head()

,OBJECTID,CODIGO,TIPO,NOME,FONTE,CD_REGIONA,NM_REGIONA,SHAPE_AREA,SHAPE_LEN,geometry
0,20,9.00,DIVISA DE BAIRROS,ÁGUA VERDE,Decreto Municipal 774 / 1975,7.00,REGIONAL PORTÃO,4773046.30,9760.92,"POLYGON ((673879.287 7183803.691, 673907.541 7..."
1,22,18.00,DIVISA DE BAIRROS,JARDIM SOCIAL,Decreto Municipal 774 / 1975,1.00,REGIONAL MATRIZ,1958421.84,6055.95,"POLYGON ((678071.018 7187798.489, 678071.550 7..."
2,23,57.00,DIVISA DE BAIRROS,XAXIM,Decreto Municipal 774 / 1975,2.00,REGIONAL BOQUEIRÃO,9085390.75,13783.59,"POLYGON ((674448.852 7179898.885, 674532.900 7..."
3,5,22.00,DIVISA DE BAIRROS,JARDIM DAS AMÉRICAS,Decreto Municipal 774 / 1975,3.00,REGIONAL CAJURU,3859033.54,8818.50,"POLYGON ((678209.542 7181641.525, 678107.870 7..."
4,9,26.00,DIVISA DE BAIRROS,GUAÍRA,Decreto Municipal 774 / 1975,7.00,REGIONAL PORTÃO,2311397.49,6351.96,"POLYGON ((672709.614 7181167.829, 672599.420 7..."


In [ ]:
db['bairro'] = db['NOME'].str.capitalize().apply(unidecode.unidecode)

In [ ]:
db['bairro'].head()

0             Agua verde
1          Jardim social
2                  Xaxim
3    Jardim das americas
4                 Guaira
Name: bairro, dtype: object

In [ ]:
db['bairro'] = db['bairro'].replace('Cidade industrial de curitiba', 'Cidade industrial')

In [ ]:
merged = db.set_index('bairro').join(da_price.set_index('bairro'))
merged = merged.reset_index()
#merged = merged.fillna(0)

In [ ]:
merged['precos'] = merged['precos'].fillna(merged.groupby('NM_REGIONA')['precos'].transform('median'))

In [ ]:
merged.head()

,bairro,OBJECTID,CODIGO,TIPO,NOME,FONTE,CD_REGIONA,NM_REGIONA,SHAPE_AREA,SHAPE_LEN,geometry,precos
0,Agua verde,20,9.00,DIVISA DE BAIRROS,ÁGUA VERDE,Decreto Municipal 774 / 1975,7.00,REGIONAL PORTÃO,4773046.30,9760.92,"POLYGON ((673879.287 7183803.691, 673907.541 7...",1500.00
1,Jardim social,22,18.00,DIVISA DE BAIRROS,JARDIM SOCIAL,Decreto Municipal 774 / 1975,1.00,REGIONAL MATRIZ,1958421.84,6055.95,"POLYGON ((678071.018 7187798.489, 678071.550 7...",1300.00
2,Xaxim,23,57.00,DIVISA DE BAIRROS,XAXIM,Decreto Municipal 774 / 1975,2.00,REGIONAL BOQUEIRÃO,9085390.75,13783.59,"POLYGON ((674448.852 7179898.885, 674532.900 7...",980.00
3,Jardim das americas,5,22.00,DIVISA DE BAIRROS,JARDIM DAS AMÉRICAS,Decreto Municipal 774 / 1975,3.00,REGIONAL CAJURU,3859033.54,8818.50,"POLYGON ((678209.542 7181641.525, 678107.870 7...",1950.00
4,Guaira,9,26.00,DIVISA DE BAIRROS,GUAÍRA,Decreto Municipal 774 / 1975,7.00,REGIONAL PORTÃO,2311397.49,6351.96,"POLYGON ((672709.614 7181167.829, 672599.420 7...",1200.00


In [ ]:
 merged.groupby(['bairro']).median().sort_values('precos', ascending=True)

,OBJECTID,CODIGO,CD_REGIONA,SHAPE_AREA,SHAPE_LEN,precos
bairro,,,,,,
Guabirotuba,6,23.00,3.00,2654431.08,6576.16,450.00
Umbara,50,73.00,10.00,22449467.55,26556.82,616.00
Sitio cercado,51,65.00,10.00,11199570.52,14105.87,616.00
Ganchinho,53,72.00,10.00,11359302.45,22063.15,616.00
Tatuquara,49,74.00,12.00,11234800.38,21895.61,675.00
...,...,...,...,...,...,...
Vila izabel,11,28.00,7.00,1216541.77,4590.33,1745.00
Batel,39,10.00,1.00,1737732.42,5622.22,1900.00
Jardim das americas,5,22.00,3.00,3859033.54,8818.50,1950.00


In [ ]:
 merged.groupby(['NM_REGIONA']).median().sort_values('precos', ascending=False)

,OBJECTID,CODIGO,CD_REGIONA,SHAPE_AREA,SHAPE_LEN,precos
NM_REGIONA,,,,,,
REGIONAL MATRIZ,23.50,10.50,1.00,1876561.14,6225.21,1475.00
REGIONAL CAJURU,30.00,21.50,3.00,4014396.44,9308.81,1250.00
REGIONAL SANTA FELICIDADE,69.00,48.00,5.00,3512540.74,10446.54,1200.00
REGIONAL PORTÃO,10.50,27.50,7.00,2284859.19,6949.10,1089.00
REGIONAL BOA VISTA,48.50,49.50,4.00,4237817.97,11098.51,1047.50
REGIONAL CIC,44.50,68.50,11.00,8355042.00,18813.55,950.00
REGIONAL PINHEIRINHO,28.00,41.00,8.00,5049840.13,11816.64,940.00
REGIONAL BOQUEIRÃO,34.00,56.50,2.00,10536322.74,16155.39,837.50
REGIONAL SANTA FELICIDADE / REGIONAL PORTÃO,33.00,44.00,5.00,8569837.61,16637.62,800.00


In [ ]:
merged.head()

,bairro,OBJECTID,CODIGO,TIPO,NOME,FONTE,CD_REGIONA,NM_REGIONA,SHAPE_AREA,SHAPE_LEN,geometry,precos
0,Agua verde,20,9.00,DIVISA DE BAIRROS,ÁGUA VERDE,Decreto Municipal 774 / 1975,7.00,REGIONAL PORTÃO,4773046.30,9760.92,"POLYGON ((673879.287 7183803.691, 673907.541 7...",1500.00
1,Jardim social,22,18.00,DIVISA DE BAIRROS,JARDIM SOCIAL,Decreto Municipal 774 / 1975,1.00,REGIONAL MATRIZ,1958421.84,6055.95,"POLYGON ((678071.018 7187798.489, 678071.550 7...",1300.00
2,Xaxim,23,57.00,DIVISA DE BAIRROS,XAXIM,Decreto Municipal 774 / 1975,2.00,REGIONAL BOQUEIRÃO,9085390.75,13783.59,"POLYGON ((674448.852 7179898.885, 674532.900 7...",980.00
3,Jardim das americas,5,22.00,DIVISA DE BAIRROS,JARDIM DAS AMÉRICAS,Decreto Municipal 774 / 1975,3.00,REGIONAL CAJURU,3859033.54,8818.50,"POLYGON ((678209.542 7181641.525, 678107.870 7...",1950.00
4,Guaira,9,26.00,DIVISA DE BAIRROS,GUAÍRA,Decreto Municipal 774 / 1975,7.00,REGIONAL PORTÃO,2311397.49,6351.96,"POLYGON ((672709.614 7181167.829, 672599.420 7...",1200.00


In [ ]:
merged = merged.to_crs(epsg=4326) # convert the coordinate reference system to lat/long
lga_json = merged.__geo_interface__ #covert to geoJSON

In [ ]:
MAPBOX_ACCESSTOKEN = 'pk.eyJ1IjoiZm1hcmNlbG5ldmVzIiwiYSI6ImNranN3NDkzNTcxYmIzM255dzEzMzA0bm4ifQ.Exq1iIAFQfPb3_sSTMiDZw'

zmin = 450
zmax = 2300

# Set the data for the map
data = go.Choroplethmapbox(
        geojson = lga_json,             #this is your GeoJSON
        locations = merged.index,    #the index of this dataframe should align with the 'id' element in your geojson
        z = merged.precos, #sets the color value
        text = merged.bairro,    #sets text for each shape
        colorbar=dict(thickness=20, ticklen=3,outlinewidth=0), #adjusts the format of the colorbar
        marker_line_width=1, marker_opacity=0.7, colorscale="Viridis", #adjust format of the plot
        zmin=zmin, zmax=zmax,           #sets min and max of the colorbar
        hovertemplate = "<b>%{text}</b><br>" +
                    "%{z}<br>" +
                    "<extra></extra>")  # sets the format of the text shown when you hover over each shape

# Set the layout for the map
layout = go.Layout(
    title = {'text': f"Apartment Rent prices, Curitiba-PR",
            'font': {'size':24}},       #format the plot title
    mapbox1 = dict(
        domain = {'x': [0, 1],'y': [0, 1]}, 
        center = dict(lat=-25.5139519 , lon=-49.3883419),
        accesstoken = MAPBOX_ACCESSTOKEN, 
        zoom = 10),                      
    autosize=True,
    height=650,
    margin=dict(l=0, r=0, t=40, b=0))

# Generate the map
fig=go.Figure(data=data, layout=layout)
fig.show()